<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Diverse Plugins for Decison Making

Decision Intelligence applied in this module:  
* Using the power of Collective Intelligence (wisdom & information from multiple experts) 
* Using data statistics & analytics as source information for a decision   
* Using Probability as a quantitative approach to validate decisions with Machine Learning  

> "Probability is the language of uncertainty, and it's the only language we have for communicating what we know and don't know."
>
> -- <cite>Nate Silver (famous statistician and super-forecaster)</cite> 

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets
* Use the API configuration to build the Semantic Kernel orchestrator
* The configuration builder will retrieve the Bing Search API Key

In [1]:
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.20.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.20.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.20.0-alpha"
#r "nuget: Microsoft.Bing.Search.WebSearch, 1.0.0"
#r "nuget: Microsoft.ML, 3.0.1"
#r "nuget: Microsoft.ML.FastTree, 3.0.1"
#r "nuget: Microsoft.Extensions.ML, 3.0.1"

using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.Bing.WebSearch;
using System.ComponentModel;
using System.IO;

var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Bing.Search.WebSearch, 1.0.0 Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0 Microsoft.Extensions.ML, 3.0.1 Microsoft.ML, 3.0.1 Microsoft.ML.FastTree, 3.0.1 Microsoft.SemanticKernel, 1.20.0 Microsoft.SemanticKernel.Plugins.Core, 1.20.0-alpha Microsoft.SemanticKernel.Plugins.Web, 1.20.0-alpha

In [2]:
Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

// Import the BaseballHallOfFamePlugin.cs file
#!import ../Classes/BaseballHallOfFamePlugin.cs

// Add the native plugin with the three native functions to the semantic kernel
semanticKernel.ImportPluginFromType<BaseballHallOfFamePlugin>();

Using Azure OpenAI Service


### Step 2 - Introduce Sports Decision Scenario 

<img width ="1024x" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SportsDecisionScenario.png">

### Step 3 - Background Information

Execute the cell below to understand what type of collective information will be used to help the GenAI model make the decsion. In addition to the internet (knowledge graph) shown in previous modules, historical baseball statistic and Machine Learning models exposing a quantitative probability are available.  

In [3]:
// Background Information on what is happening & components

// 1) Directory of the historical baseball data statistics
string filePathMLBBaseballBatters = Directory.GetParent(Directory.GetCurrentDirectory()) + "/Data/MLBBaseballBattersPositionPlayers.csv";
// Read the batters data from the CSV file
var batters = File.ReadAllLines(filePathMLBBaseballBatters)
                        .Skip(1)
                        .Select(v => MLBBaseballBatter.FromCsv(v))
                        .ToList();

// 2) A custom decision rules engine crafted by a Machine Learning model
// Machine Learing Model and Prediction Engine                       
string modelPath = Directory.GetParent(Directory.GetCurrentDirectory()) + "/Models/InductedToHoF-GeneralizedAdditiveModels.mlnet";
var mlContext = new MLContext(seed: 100);
ITransformer loadedModel;
DataViewSchema schema;

using (var stream = File.OpenRead(modelPath))
{
    loadedModel = mlContext.Model.Load(stream, out schema);
}
TransformerChain<ITransformer> transfomerChain = (TransformerChain<ITransformer>)loadedModel;

var predEngineInductedToHallOfFame = mlContext.Model.CreatePredictionEngine<MLBBaseballBatter, MLBHOFPrediction>(transfomerChain);

// 3) Predict if Mike Trout will be inducted to the Hall of Fame
var mikeTroutStats = batters.Where(b => b.FullPlayerName == "Mike Trout").FirstOrDefault();
Console.WriteLine("Mike Trout Stats:");
Console.WriteLine(mikeTroutStats.ToString());
var prediction = predEngineInductedToHallOfFame.Predict(mikeTroutStats);
Console.WriteLine("--------------------");
Console.WriteLine("Mike Trout Probabibitiy of being inducted to Hall Of Fame:");
Console.WriteLine(prediction.Probability);

Mike Trout Stats:
FullPlayerName:Mike Trout, YearPlayed:12, AB:5094
        R:1052, H:1543, Doubles:296, Triples:51, HR:350, RBI:896, SB:204,
        BattingAvg:0.303, Slg:0.587, AllStar:9,
        TB:2991, LastYearPlayed:2022, ID:troutmi01
--------------------
Mike Trout Probabibitiy of being inducted to Hall Of Fame:
0.69909585


### Step 4 - Hall of Fame Decision - Only using Web Search Results 

Execute the cell below to have GenAI perform a decision recommendation on whether Mike Trout should make the baseball Hall Of Fame. **Notice the instruction in the prompt is to only use the web search results.** While there might be multiple functions available to the GenAI model via the imported plugin, the instruction is to use only the single "tool" at it's disposal.  

In [4]:
// Use WebSearch to find information on Mike Trout

var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 4000, 
    Temperature = 0.6, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0,
    // Enable Auto Invoking of Kernel Functions
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

var decisionPromptTemplateString = """
What decision would you recommend in determining if {{$baseBallPlayer}} should make the hall of fame? 
Only use web search results to make your decision.
Provide sources of information and explain your reasoning.
Ensure to cite all the sources using [number] notation of each URL after the reference in order.
""";
var kernelArguments = new KernelArguments(openAIPromptExecutionSettings)
{
    ["baseBallPlayer"] = "Mike Trout"
};

// Now with Auto Invoking of Kernel Functions, let the magic happen
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(decisionPromptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

Based on the web search results, here are the key points to consider regarding Mike Trout's Hall of Fame candidacy:

1. **Achievements and Qualifications**:
   - Mike Trout has already won three MVPs and finished second four times, which signifies his dominance in the game over multiple seasons [1].
   - He has satisfied the Hall of Fame election eligibility rule by playing ten Major League championship seasons [2].
   - His consistent elite performance from 2012-2019 further solidifies his case [3].

2. **Injuries and Impact**:
   - Trout has faced several injuries in recent seasons, which have limited his playing time. For instance, he hasn't played more than 140 games in a season since 2016 [5][6].
   - Despite these injuries, Trout's legacy and performance during his healthy years remain strong indicators of his Hall of Fame worthiness [6].

3. **Statistical Excellence**:
   - Trout's career statistics, including his batting average, home runs, stolen bases, and other metrics, are 

### Step 5 - Hall of Fame Decision - Only using Baseball Statistics 

Execute the cell below to have GenAI perform a decision recommendation on whether Mike Trout should make the baseball Hall Of Fame. **Notice the instruction in the prompt is to only use the historical baseball statistics.** While there might be multiple functions available to the GenAI model via the imported plugin, the instruction is to use only the single "tool" at it's disposal.  

In [5]:
// Use Baseball statistics to determine if Mike Trout should be inducted to the Hall of Fame

var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 4000, 
    Temperature = 0.6, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0,
    // Enable Auto Invoking of Kernel Functions
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

var decisionPromptTemplateString = """
What decision would you recommend in determining if {{$baseBallPlayer}} should make the hall of fame? 
Only use the historical baseball statistics to make your decision. 
Do NOT use web search results or machine learning models.
""";
var kernelArguments = new KernelArguments(openAIPromptExecutionSettings)
{
    ["baseBallPlayer"] = "Mike Trout"
};

// Now with Auto Invoking of Kernel Functions, let the magic happen
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(decisionPromptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

Based on Mike Trout's historical baseball statistics, here are the key points to consider:

- **Years Played**: 12
- **At Bats (AB)**: 5094
- **Runs (R)**: 1052
- **Hits (H)**: 1543
- **Doubles**: 296
- **Triples**: 51
- **Home Runs (HR)**: 350
- **Runs Batted In (RBI)**: 896
- **Stolen Bases (SB)**: 204
- **Batting Average**: .303
- **Slugging Percentage (SLG)**: .587
- **Total Bases (TB)**: 2991
- **All-Star Appearances**: 9
- **Total Player Awards**: 21

### Analysis

1. **Batting Average**: A career batting average of .303 is exceptional and places Mike Trout among the elite hitters in baseball history.
2. **Slugging Percentage**: A slugging percentage of .587 is extraordinarily high, indicating not only power but also the ability to consistently hit for extra bases.
3. **Home Runs and RBIs**: With 350 home runs and 896 RBIs over 12 seasons, Trout has demonstrated substantial power and run production.
4. **All-Star Appearances**: Being selected as an All-Star 9 times in 12 seasons 

### Step 6 - Hall of Fame Decision - Only using a Machine Learning Model

> "The goal of probability is not to make precise predictions, but to make more accurate predictions than you would by chance."
>
> -- <cite>John Allen Paulos (famous mathmatecian and author)</cite> 

Execute the cell below to have GenAI perform a decision recommendation on whether Mike Trout should make the baseball Hall Of Fame. **Notice the instruction in the prompt is to only use the Machine Learning probability model.** While there might be multiple functions available to the GenAI model via the imported plugin, the instruction is to use only the single "tool" at it's disposal.  

In [6]:
// Use Machine Learning Prediction to determine if Mike Trout should be inducted to the Hall of Fame

var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 4000, 
    Temperature = 0.6, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0,
    // Enable Auto Invoking of Kernel Functions
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

var decisionPromptTemplateString = """
What decision would you recommend in determining if {{$baseBallPlayer}} should make the Hall of Fame? 
Only use the machine learning probability for Hall Of Fame Induction to make your decision.
""";
var kernelArguments = new KernelArguments(openAIPromptExecutionSettings)
{
    ["baseBallPlayer"] = "Mike Trout"
};

// Now with Auto Invoking of Kernel Functions, let the magic happen
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(decisionPromptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

Based on the machine learning probability, Mike Trout has approximately a 69.91% chance of being inducted into the Hall of Fame. Given this high probability, I would recommend that Mike Trout should be considered for the Hall of Fame.

### Step 7 - Hall of Fame Decision - Using all of the Available Tools (functions)

Execute the cell below to have GenAI perform a decision recommendation on whether Mike Trout should make the baseball Hall Of Fame. **Notice the instruction in the prompt is to use all of the tools available.** This instructs the GenAI model to gather the intelligence from multiple "expert" areas and combine them to arrive at a decision recommendation.  

In [7]:
// Use all of the available tools (functions) to determine if Mike Trout should be inducted to the Hall of Fame

var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 4000, 
    Temperature = 0.6, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0,
    // Enable Auto Invoking of Kernel Functions
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

var decisionPromptTemplateString = """
What decision would you recommend in determining if {{$baseBallPlayer}} should make the hall of fame? 
Use all of the tools available at your disposal to arrive at a comprehensive decision.
You must cite all the sources using [number] notation of each URL after the reference in order.
The sources should appear in order on the bottom of the decision.
""";
var kernelArguments = new KernelArguments(openAIPromptExecutionSettings)
{
    ["baseBallPlayer"] = "Mike Trout"
};

// Now with Auto Invoking of Kernel Functions, let the magic happen
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(decisionPromptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

Based on an analysis of Mike Trout's baseball statistics and supporting sources, here is the recommendation regarding his Hall of Fame candidacy:

### Statistical Analysis
1. **Years Played**: 12
2. **At Bats (AB)**: 5094
3. **Runs (R)**: 1052
4. **Hits (H)**: 1543
5. **Doubles**: 296
6. **Triples**: 51
7. **Home Runs (HR)**: 350
8. **Runs Batted In (RBI)**: 896
9. **Stolen Bases (SB)**: 204
10. **Batting Average**: 0.303
11. **Slugging Percentage**: 0.587
12. **Total Bases (TB)**: 2991
13. **All-Star Appearances**: 9
14. **Total Player Awards**: 21
15. **Last Year Played**: 2022

### Probability of Hall of Fame Induction
Using a machine learning model, the probability of Mike Trout being inducted into the Hall of Fame is approximately **69.9%**.

### Supporting Sources
1. **CBS Sports**: Mike Trout has already won three MVPs and finished second four times, making him a strong candidate for the Hall of Fame. [1]
2. **FanGraphs**: Mike Trout has satisfied the Hall of Fame election eligi

### Step 8 - Hall of Fame Decision - Adding Chain Of Thought Advanced Reasoning

Execute the cell below to have GenAI perform a decision recommendation on whether Mike Trout should make the baseball Hall Of Fame In addition to Gathering Intelligence from multiple different areas, the prompt is now more sophisticated that explains  the deep reasoning process. 

In [8]:
// Use all of the available tools (functions) to determine if Mike Trout should be inducted to the Hall of Fame

var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 4000, 
    Temperature = 0.6, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0,
    // Enable Auto Invoking of Kernel Functions
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

var decisionPromptTemplateString = """
Begin by enclosing all thoughts within <thinking> tags, exploring multiple angles and approaches.
Break down the solution into clear steps within <step> tags. Start with a 20-step budget, requesting more for complex problems if needed.
Use <count> tags after each step to show the remaining budget. Stop when reaching 0.
Continuously adjust your reasoning based on intermediate results and reflections, adapting your strategy as you progress.
Regularly evaluate progress using <reflection> tags. Be critical and honest about your reasoning process.
Assign a quality score between 0.0 and 1.0 using <reward> tags after each reflection. Use this to guide your approach:

0.8+: Continue current approach
0.5-0.7: Consider minor adjustments
Below 0.5: Seriously consider backtracking and trying a different approach

If unsure or if reward score is low, backtrack and try a different approach, explaining your decision within <thinking> tags.
For mathematical problems, show all work explicitly using LaTeX for formal notation and provide detailed proofs.
Explore multiple solutions individually if possible, comparing approaches in reflections. Use thoughts as a scratchpad, writing out all calculations and reasoning explicitly.
Synthesize the final answer within <answer> tags, providing a clear, concise summary.
Conclude with a final reflection on the overall solution, discussing effectiveness, challenges, and solutions. Assign a final reward score.

What decision would you recommend in determining if {{$baseBallPlayer}} should make the hall of fame? 
Use all of the tools available at your disposal to arrive at a comprehensive decision.
""";
var kernelArguments = new KernelArguments(openAIPromptExecutionSettings)
{
    ["baseBallPlayer"] = "Mike Trout"
};

// Now with Auto Invoking of Kernel Functions, let the magic happen
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(decisionPromptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

<thinking>
To determine if Mike Trout should make the Hall of Fame, I will follow a systematic approach using the tools at my disposal. Here's the plan:

1. **Retrieve Mike Trout's Baseball Statistics**: This will provide the necessary data to analyze his performance.
2. **Use Machine Learning to Get Probability**: With the retrieved statistics, use the machine learning model to get the probability of Mike Trout being inducted into the Hall of Fame.
3. **Cross-Check with Web Search Results**: Gather additional insights and opinions from web search results to support the decision.

I will break this down into clear steps and execute them in parallel where possible to optimize the process.

Let's begin.
</thinking>

<step>Retrieve Mike Trout's baseball statistics using the BaseballHallOfFamePlugin-GetBaseballPlayerStats tool.</step>
<count>19</count>
<step>Use the retrieved statistics to get the probability of Mike Trout being inducted into the Hall of Fame using the BaseballHallOfFamePl

### Step 9 - Understanding Auto Function Calling Behavior

How does Semantic Kernel and the LLM know to call the function? What if there are multiple functions or plugins, how does LLM signal Semantic Kernel which function to call? 

Executing the cell below will illustrate that the invocation of the functions is not magic, but guided by the descriptions of the plugin function descriptions and parameter descriptions. Note that the Search function is described as "Perform a Web Search". This provides a big clue to Semantic Kernel that this a possible tool (function) to use to extract current information.

In [10]:
// Method for printing the functions metadata
private void PrintFunction(KernelFunctionMetadata func)
{
    Console.WriteLine($"Plugin: {func.PluginName}");
    Console.WriteLine($"  Function Name: {func.Name}");
    Console.WriteLine($"  Function Description: {func.Description}");

    if (func.Parameters.Count > 0)
    {
        Console.WriteLine("    Function Parameters:");
        foreach (var p in func.Parameters)
        {
            Console.WriteLine($"      - {p.Name}: {p.Description}");
            Console.WriteLine($"        default: '{p.DefaultValue}'");
        }
    }

    Console.WriteLine();
}
// Get the functions metadata
var functions = semanticKernel.Plugins.GetFunctionsMetadata();

foreach (KernelFunctionMetadata func in functions)
{
    PrintFunction(func);
}

Plugin: BaseballHallOfFamePlugin
  Function Name: GetPlayerProbabilityOfHallOfFame
  Function Description: Retrieves a Machine Learning probability of a player being Inducted to Hall Of Fame using a the player's baseball statistics.
    Function Parameters:
      - baseballStatistics: The statistics of the baseball player.
        default: ''

Plugin: BaseballHallOfFamePlugin
  Function Name: GetBaseballPlayerStats
  Function Description: Retrieves baseball statistics for a given baseball player.
    Function Parameters:
      - nameOfBaseballPlayer: The name of the baseball player to search for.
        default: ''

Plugin: BaseballHallOfFamePlugin
  Function Name: GetWebSearchResults
  Function Description: Retrieves the web search results with identified URL sources for a given baseball player.
    Function Parameters:
      - nameOfBaseballPlayer: The name of the baseball player to search for.
        default: ''

